# Ground Truth Planning Model: NSRTs and Motion Primitives

In this notebook, we'll explore the ground truth planning model for the Satellites domain. This includes:

1. **Motion Primitives (Options)**: Low-level controllers that execute actions
2. **NSRTs**: High-level symbolic operators that use predicates and options
3. **Bilevel Planning Process**: How high-level and low-level planning work together

Understanding this ground truth model is crucial because **this is what IVNTR learns to approximate** using neural predicates!

## Key Insight
The ground truth model represents the **"oracle knowledge"** that a perfect planner would have. IVNTR's goal is to learn this knowledge from demonstrations, replacing some predicates with neural networks while maintaining the same planning capabilities.

In [1]:
import sys
import numpy as np

# Add the project root to path
sys.path.append('..')

# Import IVNTR components
from predicators.envs.satellites import SatellitesEnv
from predicators.ground_truth_models.satellites.nsrts import SatellitesGroundTruthNSRTFactory
from predicators.ground_truth_models.satellites.options import SatellitesGroundTruthOptionFactory
from predicators import utils
from predicators.settings import CFG

# Configure for tutorial
utils.reset_config({
    "env": "satellites",
})

# Configure for tutorial
CFG.seed = 42
CFG.num_train_tasks = 3
CFG.satellites_num_sat_train = [2, 3]
CFG.satellites_num_obj_train = [2, 3]

# Create environment and get a sample task
env = SatellitesEnv(use_gui=False)
train_tasks = env.get_train_tasks()
sample_task = train_tasks[0]

print("✅ Setup complete! Environment and sample task ready.")

/Users/libowen/opt/anaconda3/envs/ivntr/lib/python3.8/site-packages/bosdyn/client/sdk.py:17: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/Users/libowen/opt/anaconda3/envs/ivntr/lib/python3.8/site-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/libowen/opt/anaconda3/envs/ivntr/lib/python3.8/site-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.ai')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest

✅ Setup complete! Environment and sample task ready.


## 1. Motion Primitives (Options)

Options are parameterized low-level controllers that bridge symbolic planning and continuous control. Each option:

- **Takes objects as arguments** (e.g., which satellite and object to act on)
- **Takes continuous parameters** (e.g., target positions, sampled by NSRTs)
- **Produces low-level actions** (e.g., the 10-dimensional action vectors)

In the Satellites domain, we have 8 different options corresponding to different types of actions.

In [2]:
# Load ground truth options
types_dict = {t.name: t for t in env.types}
predicates_dict = {p.name: p for p in env.predicates}

options = SatellitesGroundTruthOptionFactory.get_options(
    "satellites", types_dict, predicates_dict, env.action_space
)
options_dict = {opt.name: opt for opt in options}

print("🎯 Ground Truth Options (Motion Primitives):\n")

for opt_name, opt in sorted(options_dict.items()):
    types_str = ", ".join([t.name for t in opt.types])
    param_space = getattr(opt, 'params_space', 'No parameters')
    
    print(f"📦 {opt_name}({types_str})")
    if param_space.shape[0] > 0:
        print(f"   Parameters: {param_space.shape[0]}D continuous vector")
        print(f"   Bounds: [{param_space.low[0]:.1f}, {param_space.high[0]:.1f}]")
    else:
        print(f"   Parameters: {param_space}")
    print()

print("💡 Key Insight: Options abstract away the details of how to execute actions,")
print("   letting the symbolic planner focus on WHAT to do, not HOW to do it.")

🎯 Ground Truth Options (Motion Primitives):

📦 Calibrate(satellite, object)
   Parameters: Box([], [], (0,), float32)

📦 MoveAway(satellite, object)
   Parameters: 2D continuous vector
   Bounds: [0.0, 1.0]

📦 MoveTo(satellite, object)
   Parameters: 2D continuous vector
   Bounds: [0.0, 1.0]

📦 ShootChemX(satellite, object)
   Parameters: Box([], [], (0,), float32)

📦 ShootChemY(satellite, object)
   Parameters: Box([], [], (0,), float32)

📦 UseCamera(satellite, object)
   Parameters: Box([], [], (0,), float32)

📦 UseGeiger(satellite, object)
   Parameters: Box([], [], (0,), float32)

📦 UseInfraRed(satellite, object)
   Parameters: Box([], [], (0,), float32)

💡 Key Insight: Options abstract away the details of how to execute actions,
   letting the symbolic planner focus on WHAT to do, not HOW to do it.


### Understanding Option Policies

Let's examine how options work by looking at their policies. Each option has a policy that maps:
- **State** + **Objects** + **Parameters** → **Action**

This is the "motion primitive" - a reusable piece of control logic.

In [3]:
# Let's examine specific option policies
state = sample_task.init
satellites = list(state.get_objects(env._sat_type))
objects = list(state.get_objects(env._obj_type))

sat = satellites[0]
obj = objects[0]

print(f"🛸 Using satellite: {sat.name}")
print(f"🎯 Using object: {obj.name}")
print(f"Initial satellite position: ({state.get(sat, 'x'):.3f}, {state.get(sat, 'y'):.3f})")
print(f"Object position: ({state.get(obj, 'x'):.3f}, {state.get(obj, 'y'):.3f})\n")

# 1. MoveTo option - requires 2D target position parameters
moveto_option = options_dict["MoveTo"]
target_params = np.array([0.5, 0.5])  # Move to center

print("🚀 MoveTo Option:")
print(f"   Target parameters: {target_params}")

# Execute the policy to get an action
moveto_action = moveto_option.policy(state, {}, [sat, obj], target_params)
print(f"   Generated action: {moveto_action.arr}")
print(f"   Action breakdown:")
print(f"     - Current sat pos: ({moveto_action.arr[0]:.3f}, {moveto_action.arr[1]:.3f})")
print(f"     - Object pos: ({moveto_action.arr[2]:.3f}, {moveto_action.arr[3]:.3f})")
print(f"     - Target sat pos: ({moveto_action.arr[4]:.3f}, {moveto_action.arr[5]:.3f})")
print(f"     - Action flags: {moveto_action.arr[6:]}")

# 2. Calibrate option - no parameters needed
calibrate_option = options_dict["Calibrate"]
print(f"\n🔧 Calibrate Option:")
print(f"   Parameters needed: None (uses current positions)")

calibrate_action = calibrate_option.policy(state, {}, [sat, obj], np.array([]))
print(f"   Generated action: {calibrate_action.arr}")
print(f"   Note: calibrate flag (index 6) = {calibrate_action.arr[6]} (set to 1.0)")

# 3. ShootChemX option - no parameters
shoot_option = options_dict["ShootChemX"]
print(f"\n💥 ShootChemX Option:")
shoot_action = shoot_option.policy(state, {}, [sat, obj], np.array([]))
print(f"   Generated action: {shoot_action.arr}")
print(f"   Note: shoot_chem_x flag (index 7) = {shoot_action.arr[7]} (set to 1.0)")

print(f"\n💡 Pattern: Each option encodes a specific type of action by setting the right flags")
print(f"   and using appropriate parameter interpretation.")

🛸 Using satellite: sat0
🎯 Using object: obj0
Initial satellite position: (0.439, 0.859)
Object position: (0.443, 0.227)

🚀 MoveTo Option:
   Target parameters: [0.5 0.5]
   Generated action: [0.43887845 0.85859793 0.4434142  0.22723871 0.5        0.5
 0.         0.         0.         0.        ]
   Action breakdown:
     - Current sat pos: (0.439, 0.859)
     - Object pos: (0.443, 0.227)
     - Target sat pos: (0.500, 0.500)
     - Action flags: [0. 0. 0. 0.]

🔧 Calibrate Option:
   Parameters needed: None (uses current positions)
   Generated action: [0.43887845 0.85859793 0.4434142  0.22723871 0.43887845 0.85859793
 1.         0.         0.         0.        ]
   Note: calibrate flag (index 6) = 1.0 (set to 1.0)

💥 ShootChemX Option:
   Generated action: [0.43887845 0.85859793 0.4434142  0.22723871 0.43887845 0.85859793
 0.         1.         0.         0.        ]
   Note: shoot_chem_x flag (index 7) = 1.0 (set to 1.0)

💡 Pattern: Each option encodes a specific type of action by set

## 2. NSRTs: Neuro-Symbolic Relational Transitions

NSRTs are the high-level symbolic operators that define the planning model. Each NSRT specifies:

- **Parameters**: Which objects it operates on
- **Preconditions**: Predicates that must be true before execution
- **Add Effects**: Predicates that become true after execution  
- **Delete Effects**: Predicates that become false after execution
- **Option**: Which motion primitive to use
- **Sampler**: How to generate continuous parameters for the option

NSRTs are the symbolic "recipes" for achieving goals in the domain.

In [4]:
# Load ground truth NSRTs
nsrts = SatellitesGroundTruthNSRTFactory.get_nsrts(
    "satellites", types_dict, predicates_dict, options_dict
)

print(f"🧠 Ground Truth NSRTs: {len(nsrts)} operators\n")

# Create a dictionary for easy access
nsrts_dict = {nsrt.name: nsrt for nsrt in nsrts}

# Display all NSRTs with their structure
for nsrt_name, nsrt in sorted(nsrts_dict.items()):
    print(f"📋 {nsrt_name}")
    print(f"   Parameters: {[str(p) for p in nsrt.parameters]}")
    print(f"   Option: {nsrt.option.name}")
    
    if nsrt.preconditions:
        print(f"   Preconditions:")
        for pre in nsrt.preconditions:
            print(f"     ✓ {pre}")
    else:
        print(f"   Preconditions: None")
    
    if nsrt.add_effects:
        print(f"   Add Effects:")
        for eff in nsrt.add_effects:
            print(f"     + {eff}")
    
    if nsrt.delete_effects:
        print(f"   Delete Effects:")
        for eff in nsrt.delete_effects:
            print(f"     - {eff}")
    
    print()

🧠 Ground Truth NSRTs: 8 operators

📋 Calibrate
   Parameters: ['?sat:satellite', '?obj:object']
   Option: Calibrate
   Preconditions:
     ✓ Sees(?sat:satellite, ?obj:object)
     ✓ CalibrationTarget(?sat:satellite, ?obj:object)
   Add Effects:
     + IsCalibrated(?sat:satellite)

📋 MoveAway
   Parameters: ['?sat:satellite', '?obj:object']
   Option: MoveAway
   Preconditions:
     ✓ Sees(?sat:satellite, ?obj:object)
   Add Effects:
     + ViewClear(?sat:satellite)
   Delete Effects:
     - Sees(?sat:satellite, ?obj:object)

📋 MoveTo
   Parameters: ['?sat:satellite', '?obj:object']
   Option: MoveTo
   Preconditions:
     ✓ ViewClear(?sat:satellite)
   Add Effects:
     + Sees(?sat:satellite, ?obj:object)
   Delete Effects:
     - ViewClear(?sat:satellite)

📋 ShootChemX
   Parameters: ['?sat:satellite', '?obj:object']
   Option: ShootChemX
   Preconditions:
     ✓ ShootsChemX(?sat:satellite)
     ✓ Sees(?sat:satellite, ?obj:object)
   Add Effects:
     + HasChemX(?obj:object)

📋 Shoot

### Detailed NSRT Analysis

Let's analyze some key NSRTs to understand how they work and why they're designed this way.

In [5]:
# Analyze key NSRTs in detail

print("🔍 Detailed NSRT Analysis\n")

# 1. MoveTo NSRT - Basic positioning
moveto_nsrt = nsrts_dict["MoveTo"]
print("1️⃣ MoveTo NSRT - Basic Positioning")
print("   Purpose: Move satellite to see objects")
print(f"   Precondition: ViewClear(?sat) - satellite's view must be unobstructed")
print(f"   Add Effect: Sees(?sat, ?obj) - satellite will see the object")
print(f"   Delete Effect: ViewClear(?sat) - view might become obstructed")
print(f"   💡 Design Logic: Only move when view is clear, movement enables seeing")
print()

# 2. Calibrate NSRT - Preparation for readings
calibrate_nsrt = nsrts_dict["Calibrate"]
print("2️⃣ Calibrate NSRT - Instrument Preparation")
print("   Purpose: Calibrate satellite's instrument")
print(f"   Preconditions:")
for pre in calibrate_nsrt.preconditions:
    print(f"     - {pre}")
print(f"   Add Effect: IsCalibrated(?sat) - satellite becomes ready for readings")
print(f"   💡 Design Logic: Must see correct calibration target, then can take readings")
print()

# 3. TakeCameraReading NSRT - Complex coordination
camera_nsrt = nsrts_dict["TakeCameraReading"]
print("3️⃣ TakeCameraReading NSRT - Complex Coordination")
print("   Purpose: Take a camera reading of an object")
print(f"   Preconditions (ALL must be true):")
for pre in camera_nsrt.preconditions:
    print(f"     - {pre}")
print(f"   Add Effect: {list(camera_nsrt.add_effects)[0]}")
print(f"   💡 Design Logic: Camera readings require ChemX preparation!")
print(f"      This creates coordination dependencies between satellites.")
print()

# 4. TakeGeigerReading NSRT - No coordination needed
geiger_nsrt = nsrts_dict["TakeGeigerReading"]
print("4️⃣ TakeGeigerReading NSRT - Independent Operation")
print("   Purpose: Take a Geiger reading (no chemicals needed)")
print(f"   Preconditions:")
for pre in geiger_nsrt.preconditions:
    print(f"     - {pre}")
print(f"   💡 Design Logic: Geiger readings don't need chemical preparation,")
print(f"      making them easier to achieve but still requiring basic setup.")
print()

print("🎯 Key Insight: NSRTs encode the domain's coordination requirements!")
print("   Different instruments have different dependencies, forcing the planner")
print("   to reason about multi-step, multi-agent coordination.")

🔍 Detailed NSRT Analysis

1️⃣ MoveTo NSRT - Basic Positioning
   Purpose: Move satellite to see objects
   Precondition: ViewClear(?sat) - satellite's view must be unobstructed
   Add Effect: Sees(?sat, ?obj) - satellite will see the object
   Delete Effect: ViewClear(?sat) - view might become obstructed
   💡 Design Logic: Only move when view is clear, movement enables seeing

2️⃣ Calibrate NSRT - Instrument Preparation
   Purpose: Calibrate satellite's instrument
   Preconditions:
     - Sees(?sat:satellite, ?obj:object)
     - CalibrationTarget(?sat:satellite, ?obj:object)
   Add Effect: IsCalibrated(?sat) - satellite becomes ready for readings
   💡 Design Logic: Must see correct calibration target, then can take readings

3️⃣ TakeCameraReading NSRT - Complex Coordination
   Purpose: Take a camera reading of an object
   Preconditions (ALL must be true):
     - Sees(?sat:satellite, ?obj:object)
     - HasCamera(?sat:satellite)
     - IsCalibrated(?sat:satellite)
     - HasChemX(?obj:

## 3. Bilevel Planning Process

Now let's demonstrate the actual bilevel planning process used in IVNTR. Planning happens in two steps:

1. **High-level Planning**: Generate symbolic skeletons (sequences of NSRTs)
2. **Low-level Refinement**: Sample continuous parameters and refine into executable options

This two-step process is what enables symbolic reasoning while handling continuous control.

For more detailed explanations about bilevel learning, checkout a recent repo we are actively developing [here](https://github.com/tomsilver/bilevel-planning). Tom has provided an exhaustive introduction.

In [6]:
# Demonstrate planning with NSRTs
print("🎯 Planning with NSRTs\n")

# Analyze current state and applicable NSRTs
print("📊 Current State Analysis:")
print(f"   Satellites: {len(satellites)}")
print(f"   Objects: {len(objects)}")
print(f"   Goals: {len(sample_task.goal)}")
print()

# Check which predicates hold initially
print("✅ Initial Predicate Status:")
for sat in satellites:
    is_calibrated = env._IsCalibrated_holds(state, [sat])
    view_clear = env._ViewClear_holds(state, [sat])
    print(f"   {sat.name}: IsCalibrated={is_calibrated}, ViewClear={view_clear}")
    
    for obj in objects:
        sees = env._Sees_holds(state, [sat, obj])
        cal_target = env._CalibrationTarget_holds(state, [sat, obj])
        if sees or cal_target:
            print(f"     → {obj.name}: Sees={sees}, CalibrationTarget={cal_target}")

for obj in objects:
    has_x = env._HasChemX_holds(state, [obj])
    has_y = env._HasChemY_holds(state, [obj])
    if has_x or has_y:
        print(f"   {obj.name}: HasChemX={has_x}, HasChemY={has_y}")
print()

# Find applicable NSRTs
print("🔍 Applicable NSRTs in Current State:")
applicable_nsrts = []

for nsrt in nsrts:
    # Try all possible object bindings
    for sat in satellites:
        for obj in objects:
            objects_list = [sat, obj]
            
            # Check if all preconditions are satisfied
            all_preconditions_met = True
            for precondition in nsrt.preconditions:
                # Ground the precondition with actual objects
                pred_name = precondition.predicate.name
                pred_args = [objects_list[i] for i in range(len(precondition.predicate.types))]
                
                # Check if predicate holds
                pred_func = getattr(env, f"_{pred_name}_holds")
                if not pred_func(state, pred_args):
                    all_preconditions_met = False
                    break
            
            if all_preconditions_met:
                applicable_nsrts.append((nsrt.name, sat.name, obj.name))

# Display applicable NSRTs
if applicable_nsrts:
    for nsrt_name, sat_name, obj_name in applicable_nsrts:
        print(f"   ✓ {nsrt_name}({sat_name}, {obj_name})")
else:
    print(f"   ❌ No NSRTs applicable (all have unmet preconditions)")
print()

# Show what we need to achieve goals
print("🏆 Goal Analysis:")
for goal_atom in sample_task.goal:
    pred_name = goal_atom.predicate.name
    obj_names = [obj.name for obj in goal_atom.objects]
    print(f"   Goal: {pred_name}({', '.join(obj_names)})")
    
    # Find which NSRT achieves this goal
    achieving_nsrt = None
    if "Camera" in pred_name:
        achieving_nsrt = "TakeCameraReading"
    elif "Infrared" in pred_name:
        achieving_nsrt = "TakeInfraredReading"
    elif "Geiger" in pred_name:
        achieving_nsrt = "TakeGeigerReading"
    
    if achieving_nsrt:
        nsrt = nsrts_dict[achieving_nsrt]
        print(f"     Achieved by: {achieving_nsrt}")
        print(f"     Requires:")
        for precond in nsrt.preconditions:
            print(f"       - {precond}")
print()

print("💡 Planning Insight: The planner needs to work backwards from goals,")
print("   figuring out which NSRTs achieve them and what preconditions those require.")
print("   This creates chains of dependencies that must be satisfied in sequence!")

🎯 Planning with NSRTs

📊 Current State Analysis:
   Satellites: 2
   Objects: 3
   Goals: 2

✅ Initial Predicate Status:
   sat0: IsCalibrated=False, ViewClear=True
     → obj1: Sees=False, CalibrationTarget=True
   sat1: IsCalibrated=False, ViewClear=True
     → obj2: Sees=False, CalibrationTarget=True

🔍 Applicable NSRTs in Current State:
   ✓ MoveTo(sat0, obj0)
   ✓ MoveTo(sat0, obj1)
   ✓ MoveTo(sat0, obj2)
   ✓ MoveTo(sat1, obj0)
   ✓ MoveTo(sat1, obj1)
   ✓ MoveTo(sat1, obj2)

🏆 Goal Analysis:
   Goal: InfraredReadingTaken(sat1, obj2)
     Achieved by: TakeInfraredReading
     Requires:
       - HasInfrared(?sat:satellite)
       - Sees(?sat:satellite, ?obj:object)
       - IsCalibrated(?sat:satellite)
       - HasChemY(?obj:object)
   Goal: CameraReadingTaken(sat0, obj2)
     Achieved by: TakeCameraReading
     Requires:
       - Sees(?sat:satellite, ?obj:object)
       - HasCamera(?sat:satellite)
       - IsCalibrated(?sat:satellite)
       - HasChemX(?obj:object)

💡 Planning I

### Bilevel Planning Architecture

Let's demonstrate the difference between symbolic and continuous levels of planning.

In [7]:
# Demonstrate the difference between symbolic and continuous levels
print("🏗️ Bilevel Planning Architecture\n")

print("📊 LEVEL 1: Symbolic/High-level Planning")
print("   Input: Initial predicates + Goal predicates")  
print("   Process: Search over NSRT sequences")
print("   Output: Skeleton (sequence of symbolic actions)")
print("   Example skeleton for CameraReadingTaken goal:")
print("     1. MoveAway(?sat, ?obj)")
print("     2. MoveTo(?sat, ?calibration_target)")  
print("     3. Calibrate(?sat, ?calibration_target)")
print("     4. MoveTo(?sat, ?goal_object)")
print("     5. ShootChemX(?chemsat, ?goal_object)")
print("     6. TakeCameraReading(?camsat, ?goal_object)")
print()

print("⚙️ LEVEL 2: Continuous/Low-level Planning")
print("   Input: Skeleton + Current state")
print("   Process: Sample continuous parameters for each NSRT")
print("   Output: Executable option sequence")
print("   Example refinement:")
print("     1. MoveAway(sat0, obj1) → MoveTo_option(sat0, obj1, params=[0.1, 0.9])")
print("     2. MoveTo(sat0, obj0) → MoveTo_option(sat0, obj0, params=[0.3, 0.4])")
print("     3. Calibrate(sat0, obj0) → Calibrate_option(sat0, obj0, params=[])")
print("     4. MoveTo(sat0, obj1) → MoveTo_option(sat0, obj1, params=[0.6, 0.7])")
print("     5. ShootChemX(sat1, obj1) → ShootChemX_option(sat1, obj1, params=[])")
print("     6. TakeCameraReading(sat0, obj1) → UseCamera_option(sat0, obj1, params=[])")
print()

print("🎯 Why This Matters for Learning:")
print("   • Symbolic level: Learns WHICH actions to take and WHEN")
print("   • Continuous level: Learns HOW to execute actions (parameter sampling)")
print("   • IVNTR must learn symbolic predicates that enable this decomposition")
print("   • Neural predicates replace ground truth predicates but maintain structure")
print()

print("🧠 The Learning Challenge:")
print("   Traditional approaches:")
print("   ❌ Pure neural: Can't handle symbolic reasoning over long horizons")
print("   ❌ Pure symbolic: Can't learn abstractions from continuous data")
print("   ✅ IVNTR: Learns neural predicates that plug into symbolic planning!")
print()

print("💡 Key Insight: The symbolic structure acts as 'scaffolding' for learning.")
print("   Instead of learning everything from scratch, neural networks only need")
print("   to learn the predicate mappings while leveraging proven planning algorithms.")

# Trace a typical planning sequence
print("\n" + "="*60)
print("📋 Typical Planning Sequence for Camera Reading\n")

# Goal: Take a camera reading
print("🎯 Goal: CameraReadingTaken(satellite, object)")
print("\n📝 Required Planning Steps:")

camera_nsrt = nsrts_dict["TakeCameraReading"]
print(f"\n5️⃣ Final Step: {camera_nsrt.name}")
print(f"   Requires: Sees + IsCalibrated + HasCamera + HasChemX")
print(f"   Effect: CameraReadingTaken ✓")

print(f"\n4️⃣ Chemical Preparation: ShootChemX")
shoot_nsrt = nsrts_dict["ShootChemX"]
print(f"   Requires: {', '.join([str(p) for p in shoot_nsrt.preconditions])}")
print(f"   Effect: HasChemX ✓")

print(f"\n3️⃣ Calibration: Calibrate")
cal_nsrt = nsrts_dict["Calibrate"]
print(f"   Requires: {', '.join([str(p) for p in cal_nsrt.preconditions])}")
print(f"   Effect: IsCalibrated ✓")

print(f"\n2️⃣ Position for Calibration: MoveTo (to calibration target)")
moveto_nsrt = nsrts_dict["MoveTo"]
print(f"   Requires: {', '.join([str(p) for p in moveto_nsrt.preconditions])}")
print(f"   Effect: Sees(satellite, calibration_target) ✓")

print(f"\n1️⃣ Clear View: MoveAway (if needed)")
moveaway_nsrt = nsrts_dict["MoveAway"]
print(f"   Requires: {', '.join([str(p) for p in moveaway_nsrt.preconditions])}")
print(f"   Effect: ViewClear ✓")

print("\n🎭 Multi-Agent Coordination:")
print("   • Different satellites may handle different steps")
print("   • Satellite with camera ≠ satellite with ChemX capability")
print("   • Planner must coordinate multiple agents in time")

print("\n🧠 Learning Challenge:")
print("   IVNTR must learn predicates like 'Sees' and 'HasChemX' from demonstrations")
print("   while maintaining this complex coordination capability!")

🏗️ Bilevel Planning Architecture

📊 LEVEL 1: Symbolic/High-level Planning
   Input: Initial predicates + Goal predicates
   Process: Search over NSRT sequences
   Output: Skeleton (sequence of symbolic actions)
   Example skeleton for CameraReadingTaken goal:
     1. MoveAway(?sat, ?obj)
     2. MoveTo(?sat, ?calibration_target)
     3. Calibrate(?sat, ?calibration_target)
     4. MoveTo(?sat, ?goal_object)
     5. ShootChemX(?chemsat, ?goal_object)
     6. TakeCameraReading(?camsat, ?goal_object)

⚙️ LEVEL 2: Continuous/Low-level Planning
   Input: Skeleton + Current state
   Process: Sample continuous parameters for each NSRT
   Output: Executable option sequence
   Example refinement:
     1. MoveAway(sat0, obj1) → MoveTo_option(sat0, obj1, params=[0.1, 0.9])
     2. MoveTo(sat0, obj0) → MoveTo_option(sat0, obj0, params=[0.3, 0.4])
     3. Calibrate(sat0, obj0) → Calibrate_option(sat0, obj0, params=[])
     4. MoveTo(sat0, obj1) → MoveTo_option(sat0, obj1, params=[0.6, 0.7])
     5.

In [8]:
# Trace a typical planning sequence
print("📋 Typical Planning Sequence for Camera Reading\n")

# Goal: Take a camera reading
print("🎯 Goal: CameraReadingTaken(satellite, object)")
print("\n📝 Required Planning Steps:")

camera_nsrt = nsrts_dict["TakeCameraReading"]
print(f"\n5️⃣ Final Step: {camera_nsrt.name}")
print(f"   Requires: Sees + IsCalibrated + HasCamera + HasChemX")
print(f"   Effect: CameraReadingTaken ✓")

print(f"\n4️⃣ Chemical Preparation: ShootChemX")
shoot_nsrt = nsrts_dict["ShootChemX"]
print(f"   Requires: {', '.join([str(p) for p in shoot_nsrt.preconditions])}")
print(f"   Effect: HasChemX ✓")

print(f"\n3️⃣ Calibration: Calibrate")
cal_nsrt = nsrts_dict["Calibrate"]
print(f"   Requires: {', '.join([str(p) for p in cal_nsrt.preconditions])}")
print(f"   Effect: IsCalibrated ✓")

print(f"\n2️⃣ Position for Calibration: MoveTo (to calibration target)")
moveto_nsrt = nsrts_dict["MoveTo"]
print(f"   Requires: {', '.join([str(p) for p in moveto_nsrt.preconditions])}")
print(f"   Effect: Sees(satellite, calibration_target) ✓")

print(f"\n1️⃣ Clear View: MoveAway (if needed)")
moveaway_nsrt = nsrts_dict["MoveAway"]
print(f"   Requires: {', '.join([str(p) for p in moveaway_nsrt.preconditions])}")
print(f"   Effect: ViewClear ✓")

print("\n🎭 Multi-Agent Coordination:")
print("   • Different satellites may handle different steps")
print("   • Satellite with camera ≠ satellite with ChemX capability")
print("   • Planner must coordinate multiple agents in time")

print("\n🧠 Learning Challenge:")
print("   IVNTR must learn predicates like 'Sees' and 'HasChemX' from demonstrations")
print("   while maintaining this complex coordination capability!")

📋 Typical Planning Sequence for Camera Reading

🎯 Goal: CameraReadingTaken(satellite, object)

📝 Required Planning Steps:

5️⃣ Final Step: TakeCameraReading
   Requires: Sees + IsCalibrated + HasCamera + HasChemX
   Effect: CameraReadingTaken ✓

4️⃣ Chemical Preparation: ShootChemX
   Requires: ShootsChemX(?sat:satellite), Sees(?sat:satellite, ?obj:object)
   Effect: HasChemX ✓

3️⃣ Calibration: Calibrate
   Requires: Sees(?sat:satellite, ?obj:object), CalibrationTarget(?sat:satellite, ?obj:object)
   Effect: IsCalibrated ✓

2️⃣ Position for Calibration: MoveTo (to calibration target)
   Requires: ViewClear(?sat:satellite)
   Effect: Sees(satellite, calibration_target) ✓

1️⃣ Clear View: MoveAway (if needed)
   Requires: Sees(?sat:satellite, ?obj:object)
   Effect: ViewClear ✓

🎭 Multi-Agent Coordination:
   • Different satellites may handle different steps
   • Satellite with camera ≠ satellite with ChemX capability
   • Planner must coordinate multiple agents in time

🧠 Learning Chal

## Summary

In this notebook, we've explored the ground truth planning model for the Satellites domain:

### 🎯 **Motion Primitives (Options)**
- **8 different options** for different action types (MoveTo, Calibrate, Shoot chemicals, Use instruments)
- **Parameterized controllers** that bridge symbolic decisions and continuous control
- **Domain-specific encoding** of how to execute each type of action

### 🧠 **NSRTs (Symbolic Operators)**
- **8 symbolic operators** that define legal state transitions
- **Precondition-effect structure** that captures domain constraints
- **Coordination requirements** encoded through predicate dependencies

### ⚙️ **Bilevel Planning Process**
- **High-level planning**: Generate symbolic skeletons using predicate-based reasoning
- **Low-level refinement**: Sample continuous parameters and refine into executable options  
- **Two-level architecture** that separates symbolic reasoning from continuous control

### 🎯 **The IVNTR Learning Challenge**
This ground truth model represents **perfect domain knowledge**. IVNTR's challenge is to:

1. **Replace key predicates** (like `Sees`, `IsCalibrated`, `HasChemX`) with neural networks
2. **Learn from demonstrations** to predict when these predicates hold
3. **Maintain planning capability** with learned predicates
4. **Generalize to new scenarios** not seen during training

### 💡 **Key Insight: Scaffolded Learning**
The symbolic structure provides **scaffolding** for neural learning:
- Neural networks don't learn planning algorithms from scratch
- They only learn specific predicate relationships from continuous features  
- The proven bilevel planning architecture remains intact
- This enables both learning and generalization

The bilevel approach bridges the gap between symbolic reasoning (which enables long-horizon planning) and neural learning (which enables abstraction from continuous data).

---

## What's Next?

Before diving into the IVNTR learning algorithm, we need to understand:
- **What data does IVNTR learn from?** (Demonstration trajectories)
- **What must it output?** (Neural predicates that enable planning)

**Next: `03_demonstration_data_generation.ipynb` - Understanding IVNTR's Training Data**